In [ ]:
import mne
import pandas as pd 
import re #regex library to format fast to read into pd 
import os

import matplotlib.pyplot as plt
import feather 

import numpy as np
import platform

from scipy.stats import ttest_ind

from scipy import stats

from mne.channels import find_ch_adjacency, make_1020_channel_selections
from mne.stats import spatio_temporal_cluster_test
from mne.stats import (ttest_1samp_no_p, bonferroni_correction, fdr_correction,
                       
                       
                       permutation_t_test, permutation_cluster_1samp_test,spatio_temporal_cluster_1samp_test)
import sys
if platform.system()=='Darwin':
    os.chdir('/Volumes/BBC/BBC/WP1/data/EEG/tsk/')
    #sys.path.append('/Users/leupinv/BBC/WP1/data/Code/python/BBC')
    #if this doesn't work pound line right above this, restart the kernel, rerun the cell. 
    #Then uncomment the same line and rerun

else:
    #os.chdir('Z:/BBC/WP1/data/EEG/tsk')
    #sys.path.append('C:/Users/Vivi/switchdrive/BBC/WP1/data/Code/python/BBC')
    os.chdir('Z:/BBC/WP1/data/EEG/tsk')

#
#
#os.chdir('d:/switchdrive/BBC/WP1/data/Code/python/tsk_processing_1/base')
#os.chdir('c:/Users/Engi/all/BBC/WP1/data/Code/python/tsk_processing_1/base')
#os.chdir('/Users/leupinv/switchdrive/BBC/WP1/data/Code/python/tsk_processing_1/base')
from base.files_in_out import getListOfFiles,GetFiles
from stats.stats_helper import filter_list, tTest_ana, filter_list_equal

import stats.stats_helper as s_hp
import base.base_constants as cs

In [ ]:
%load_ext autoreload
%autoreload 2
import matplotlib as mpl
mpl.rcParams['svg.fonttype']='none'
mpl.rcParams['text.usetex']=False
plt.style.use('default')

mpl.rcParams['svg.fonttype']='none'
mpl.rcParams['text.usetex']=False

In [ ]:
g_num=None
eeg_format='-ave.fif'
eeg_exp='tsk'
datafolder='ana/MNE/evo_list'
 

#files = getListOfFiles(datafolder,g_num)

files = GetFiles(datafolder,g_num=g_num,eeg_format=eeg_format)

evo_files=files.fflist

In [ ]:
evo_files

In [ ]:
def crop_list(list_ev,crop_value):
    list_cropped=[]
    for cond in list_ev:
        cond_crop=[evo.crop(crop_value[0],crop_value[1]) for evo in cond]
        list_cropped.append(cond_crop)
    return list_cropped




## Figure supplementary SI 2
aware vs unaware conditions

In [ ]:
cond1=mne.read_evokeds(filter_list_equal(evo_files,f'maskON_correct_cfa_vep_norm_aware_list'))
cond2=mne.read_evokeds(filter_list_equal(evo_files,f'maskON_correct_cfa_vep_norm_unaware_list'))

In [ ]:
%matplotlib qt
FDR=True
if FDR:
    correction='FDR'
else:
    correction='noc'
pval=0.05
data_shape=[-0.1,0.5]
time_plot=[0.105,0.3,0.45]
#png=f'{rsp_cond_lab}{correction}_CACU_sysdia_{phy_cond}_{time_plot}'

ts,ps=tTest_ana([cond1,cond2],crop_value=data_shape,FDR=FDR,plot_times=time_plot,averages=0.02,p_val=0.05, topo_limits=[-6,6])

In [ ]:
#the picks define the regions to plot (they will automatically appear on the figure to know exactly their position on the EEG)
picks_P1=['D32','A11','A12','A13','A14','A24','A25','A26','A27','B8','B9','B10']



picks_VAN=['D32','A9','A10','A11','A12','A16','A15','A14','A13',
           'A22','A23','A24','A25','A29','A28','A27','A26','B6','B7','B8','B9','B10']

picks_LP=['A2','A3','A4','A19','A20','A21','A22','A32','A31','A30','A29','A5','A18',
          'A17','A16','A8','A9','A6','A7','D29','D30','D15','D16','D17','D28','D27']


picks_ERP={'P1':picks_P1,'VAN':picks_VAN,'LP':picks_LP}


In [ ]:
%matplotlib qt
cond1=mne.read_evokeds(filter_list_equal(evo_files,f'maskON_correct_cfa_vep_norm_aware_list'))
cond2=mne.read_evokeds(filter_list_equal(evo_files,f'maskON_correct_cfa_vep_norm_unaware_list'))


crop_value=[-0.1,0.5]

cond_1,cond_2=crop_list([cond1,cond2],crop_value)

evokeds = {"Aware":cond_1,

               "Unaware":cond_2,
           }






In [ ]:
%matplotlib qt

region='VAN'
picked=picks_ERP[region]

fig_fin,(ax1,ax2)=plt.subplots(2,figsize=(6,12))

fig_fin=mne.viz.plot_compare_evokeds(evokeds, combine='mean',ci=False, picks=picked,show_sensors=True,colors=['k','tab:gray'],
                                     styles={"Aware":{"linewidth":1.5},"Unaware":{"linewidth":1.5}},
                                    legend='upper right',linestyles=['--','-'],ylim=dict(eeg=[-3,6]),truncate_yaxis=False,axes=ax1)
                                

#ylim=dict(eeg=[-3,5.5])

png=f'CACU_{region}.svg'
fig_path=f'ana/results_report/images/el_waveform/{png}'

#fig_evo[0].savefig(fig_path,dpi=600,format='svg')




In [ ]:

png1='Aware - unaware'





ts=tTest_ana([cond_1,cond_2],crop_value=crop_value,FDR=False,plot_times='peaks',averages=0.02,p_val=0.05,picks=picked,png=png1,color='k',effect_size='whole',axes=ax2)


#ts=tTest_ana([cond5,cond6],crop_value=data_shape,FDR=False,plot_times='peaks',averages=0.02,p_val=0.01,picks=picks_VAN,png=png3,plot_sig_lines=False)
#ts=tTest_ana([cond7,cond8],crop_value=data_shape,FDR=False,plot_times='peaks',averages=0.02,p_val=0.01,picks=picks_VAN,png=png4)



png=f'CACU_D_{region}.svg'
fig_path=f'ana/results_report/images/el_waveform/{png}'

#fig_fin[0].savefig(fig_path,dpi=1200,format='svg')



# Figure supplementary SI 3
sys and dia removing and adding cfa 

In [ ]:
%matplotlib qt
# cfa= cfa --> with cardiac field artefact
# cfa= nc --> without cardiac field artefact
cfa='nc'
cond_1=mne.read_evokeds(filter_list(evo_files,f'maskON_correct_{cfa}_vep_norm_sys_list'))
cond_2=mne.read_evokeds(filter_list(evo_files,f'maskON_correct_{cfa}_vep_norm_dia_list'))


crop_value=[-0.1,0.5]

cond_1,cond_2=crop_list([cond_1,cond_2],crop_value)

evokeds = {"Sys":cond_1,

               "Dia":cond_2,
           }



picks_cfa=['D24','D25','D26','D29','D30','D31','D32','A8','A9','A10','A11','A12','A17','A16','A15','A14','A13',
           'A23','A24','A25','A28','A27','A26','B8','B9']


In [ ]:
%matplotlib qt

region=''
picked=picks_cfa

fig_fin,(ax1,ax2)=plt.subplots(2,figsize=(6,12))

fig_fin=mne.viz.plot_compare_evokeds(evokeds, combine='mean',ci=False, picks=picked,show_sensors=True,colors=['r','b'],
                                     styles={"Sys":{"linewidth":1.5},"Dia":{"linewidth":1.5}},
                                    legend='upper right',ylim=dict(eeg=[-3,6]),truncate_yaxis=False,axes=ax1)


#ylim=dict(eeg=[-3,5.5])

png=f'SysDia_{cfa}.svg'
fig_path=f'ana/results_report/images/el_waveform/{png}'

fig_evo[0].savefig(fig_path,dpi=600,format='svg')




## Figure 2 A, E


In [ ]:

phy_phase='exh'
if phy_phase =='inh' or phy_phase =='exh':
    time_plot=[0.75]
elif phy_phase=='sys' or phy_phase =='dia':
    time_plot=[0.105]

cond1=mne.read_evokeds(filter_list_equal(evo_files,f'maskON_correct_cfa_vep_norm_aware_{phy_phase}_list'))
cond2=mne.read_evokeds(filter_list_equal(evo_files,f'maskON_correct_cfa_vep_norm_unaware_{phy_phase}_list'))

In [ ]:
%matplotlib qt
FDR=True
if FDR:
    correction='FDR'
else:
    correction='noc'
pval=0.05
data_shape=[-0.1,0.5]

#png=f'{rsp_cond_lab}{correction}_CACU_sysdia_{phy_cond}_{time_plot}'

ts,ps=tTest_ana([cond1,cond2],crop_value=data_shape,FDR=FDR,plot_times=time_plot,averages=0.02,p_val=0.05, topo_limits=[-3,3])

## Figure 2 B C D, F G H template
set phy_sig to cardiac_phase and to rsp_phase to get two different outcomes




In [ ]:
crop_value=[-0.1,0.5]
phy_sig='rsp_phase'
rsp_cond='n'


In [ ]:
%matplotlib qt
cond_1_awa="aware_"
cond_2_awa="unaware_"
cond_3_awa="aware_"
cond_4_awa="unaware_"
if phy_sig=='rsp_phase':

    cond_1_phy="inh"
    cond_2_phy="inh"
    cond_3_phy="exh"
    cond_4_phy="exh"
elif phy_sig=='cardiac_phase':
    cond_1_phy="sys"
    cond_2_phy="sys"
    cond_3_phy="dia"
    cond_4_phy="dia"
elif phy_sig==None:
    cond_1_phy=""
    cond_2_phy=""
    cond_3_phy=""
    cond_4_phy=""
if rsp_cond=='o':
    rsp_cond_lab='tsk_o_'
elif rsp_cond=='n':
    rsp_cond_lab=''

cond_1=mne.read_evokeds(filter_list_equal(evo_files,f'{rsp_cond_lab}maskON_correct_cfa_vep_norm_{cond_1_awa}{cond_1_phy}_list'))
cond_2=mne.read_evokeds(filter_list_equal(evo_files,f'{rsp_cond_lab}maskON_correct_cfa_vep_norm_{cond_2_awa}{cond_2_phy}_list'))
cond_3=mne.read_evokeds(filter_list_equal(evo_files,f'{rsp_cond_lab}maskON_correct_cfa_vep_norm_{cond_3_awa}{cond_3_phy}_list'))
cond_4=mne.read_evokeds(filter_list_equal(evo_files,f'{rsp_cond_lab}maskON_correct_cfa_vep_norm_{cond_4_awa}{cond_4_phy}_list'))
#picks_VAN=['D32','A9','A10','A11','A12','A16','A15','A14','A13',
           #'A22','A23','A24','A25','A29','A28','A27','A26','B6','B7','B8','B9','B10']


cond_1,cond_2,cond_3,cond_4=crop_list([cond_1,cond_2,cond_3,cond_4],crop_value)

#picks=['A13','A14','A24','A25','A26','A27']
#picks=['A7','A9','A19','D29','D30','D31']
#picks=['A26','B8','B9','B10']
cond_1_lab=f"{cond_1_awa}{cond_1_phy}"
cond_2_lab=f"{cond_2_awa}{cond_2_phy}"
cond_3_lab=f"{cond_3_awa}{cond_3_phy}"
cond_4_lab=f"{cond_4_awa}{cond_4_phy}"
evokeds = {f"{cond_1_lab}":cond_1,

           f"{cond_2_lab}":cond_2,

            f"{cond_3_lab}":cond_3,

           f"{cond_4_lab}":cond_4}






## Define df  for P1 picture
the barplot is based on the P1 electrodes while the lineplots are based on the VAN

In [ ]:
region_ax1="P1"
crop_period=s_hp.define_period(region_ax1,phy_sig)
erp_df=s_hp.get_erp_df(evokeds.copy(),crop_period,picks_ERP[region_ax1])


### P1 electrodes loc
fig=plt.figure(figsize=(3,3))
ax=plt.gca()
mne.viz.plot_sensors(cond_1[0].info,axes=ax,pointsize=35)
fig_path=f'ana/results_report/images/el_waveform/P1_topo.svg'
fig.savefig(fig_path,dpi=1200,format='svg')

In [ ]:
erp_df=pd.DataFrame.from_dict(erp_df)
erp_df['id']=erp_df.index
wide1=pd.wide_to_long(erp_df,stubnames=['aware','unaware'],i='id',j=phy_sig,sep="_",suffix='\D+')
wide1.rename(columns=dict(aware='AmplitudeAware',unaware='AmplitudeUnaware'),inplace=True)
wide1.reset_index(inplace=True)
wide2=pd.wide_to_long(wide1,stubnames=['Amplitude'],i=[phy_sig,'id'],j='Awareness',sep="",suffix='\D+')
wide2.reset_index(inplace=True)
wide2

In [ ]:
import pingouin as pg
pg.rm_anova(data=wide2,dv='Amplitude',within=['Awareness',phy_sig],subject='id',effsize='np2')

In [ ]:
ttest=pg.pairwise_tests(data=wide2,dv='Amplitude',within=[phy_sig,'Awareness'],subject='id',padjust='fdr_bh')

In [ ]:
ttest

In [ ]:
#effect size cohen's D
import math
ttest['T']/math.sqrt(30)

In [ ]:
dv='Amplitude'

dict_awa_list={}



for awa in wide2['Awareness'].unique():


    awa_df=wide2[wide2['Awareness']==awa][[dv,phy_sig]]
    for phases in wide2[phy_sig].unique():
        awa_card=awa_df[awa_df[phy_sig]==phases]
        mean_card_awa=awa_card.mean()[dv]
        sem_card_awa=awa_card.sem()[dv]
        dict_awa_list[f'{awa}_{phases}']={'mean':mean_card_awa,'sem':sem_card_awa}
        #dict_awa_list.append(dict_awa)

In [ ]:
dict_awa_list

In [ ]:
%matplotlib qt
#cm=1/2.54 #cm in inches
fig_fin,((ax1,ax2),(ax3,ax4))=plt.subplots(2,2,figsize=(12,12),gridspec_kw={'width_ratios':[2,3]})
if phy_sig=='rsp_phase':
    color_list=['g','g','tab:pink','tab:pink']
    facecolor_list=['w','g','w','tab:pink']
    xticklab=['inhalation', 'exhalation']
    xlab='respiratory phase'
    cond_list=['Aware_inh','Unaware_inh','Aware_exh','Unaware_exh']
    y_lim_plot=2.5e-6
else:
    color_list=['tab:orange','tab:orange','b','b']
    facecolor_list=['w','tab:orange','w','b']
    xticklab=['systole','diastole']
    xlab='cardiac phase'
    cond_list=['Aware_sys','Unaware_sys','Aware_dia','Unaware_dia']
    y_lim_plot=6e-6


hatch_list=['///','','///','']
adj_list=[0,-0.4,-0.45,-0.85]
#color_list=['tab:orange','tab:orange','b','b']
#facecolor_list=['w','tab:orange','w','b']

alpha_list=[1,0.6,1,0.6]
x=0

for cond,color,hatch,adj,facecolor,alpha in zip(cond_list,color_list,hatch_list,adj_list,facecolor_list,alpha_list):


    fig_fin=ax3.bar(x=x+adj,height=[dict_awa_list[cond]['mean']],facecolor=facecolor,yerr=[dict_awa_list[cond]['sem']],
           edgecolor=color,hatch=hatch,linewidth=2.5,capsize=(2),ecolor='0.3',
                    width=0.5,label=cond.split('_')[0]+' '+cond.split('_')[1],alpha=alpha)
    x+=1

fig_fin=ax3.set_ylim(None,y_lim_plot)
fig_fin=ax3.set_xticks([0.3,1.85],labels=xticklab,size='x-large')
fig_fin=ax3.set_xlabel(xlab,size='x-large')
fig_fin=ax3.set_ylabel(f'{region_ax1} amplitude (uV)',size='x-large')
fig_fin=ax3.spines['top'].set_visible(False)
fig_fin=ax3.spines['right'].set_visible(False)

fig_fin=ax3.legend(bbox_to_anchor=(1,1),fontsize='large')

## Define df  for VAN picture
the barplot is based on the P1 electrodes while the lineplots are based on the VAN
unfortunately i need to reload the evoked, i cannot find a way of safely deep copy the objects, so the info structure get modified and some channels are dropped

In [ ]:
%matplotlib qt
cond_1_awa="aware_"
cond_2_awa="unaware_"

if phy_sig=='rsp_phase':
    cond_1_phy="inh"
    cond_2_phy="exh"

elif phy_sig=='cardiac_phase':
    cond_1_phy="sys"
    cond_2_phy="dia"
elif phy_sig==None:
    cond_1_phy=""
    cond_2_phy=""


cond_1=mne.read_evokeds(filter_list_equal(evo_files,f'{rsp_cond_lab}maskON_correct_cfa_vep_norm_{cond_1_awa}{cond_1_phy}_list'))
cond_2=mne.read_evokeds(filter_list_equal(evo_files,f'{rsp_cond_lab}maskON_correct_cfa_vep_norm_{cond_2_awa}{cond_1_phy}_list'))
cond_3=mne.read_evokeds(filter_list_equal(evo_files,f'{rsp_cond_lab}maskON_correct_cfa_vep_norm_{cond_1_awa}{cond_2_phy}_list'))
cond_4=mne.read_evokeds(filter_list_equal(evo_files,f'{rsp_cond_lab}maskON_correct_cfa_vep_norm_{cond_2_awa}{cond_2_phy}_list'))
#picks_VAN=['D32','A9','A10','A11','A12','A16','A15','A14','A13',
#'A22','A23','A24','A25','A29','A28','A27','A26','B6','B7','B8','B9','B10']


cond_1,cond_2,cond_3,cond_4=crop_list([cond_1,cond_2,cond_3,cond_4],crop_value)

#picks=['A13','A14','A24','A25','A26','A27']
#picks=['A7','A9','A19','D29','D30','D31']
#picks=['A26','B8','B9','B10']
cond_1_lab=f"{cond_1_awa}{cond_1_phy}"
cond_2_lab=f"{cond_2_awa}{cond_1_phy}"
cond_3_lab=f"{cond_1_awa}{cond_2_phy}"
cond_4_lab=f"{cond_2_awa}{cond_2_phy}"
evokeds = {f"{cond_1_lab}":cond_1,

           f"{cond_2_lab}":cond_2,

           f"{cond_3_lab}":cond_3,

           f"{cond_4_lab}":cond_4}





In [ ]:
region_ax2="VAN"

picked=picks_ERP[region_ax2]
if phy_sig=='rsp_phase':
    colors=['olivedrab','tab:green','tab:pink','purple']
elif phy_sig=='cardiac_phase':
    colors=['tab:orange','r','c','b']
#fig_fin,axs=plt.subplots(2,figsize=(8,12))

fig_fin=mne.viz.plot_compare_evokeds(evokeds, combine='mean',ci=False, picks=picked,show_sensors=True,colors=colors,
                                     styles={cond_1_lab:{"linewidth":1.5},cond_2_lab:{"linewidth":1.5},cond_3_lab:{"linewidth":1.5},cond_4_lab:{"linewidth":1.5}},axes=ax2,
                                    legend='upper right',linestyles=['-.','-','-.','-'],ylim=dict(eeg=[-3,6]),truncate_yaxis=False)
fig_fin=fig_fin[0]

#ylim=dict(eeg=[-3,5.5])

png=f'{rsp_cond_lab}CACU_{phy_sig}_{region_ax2}.svg'
fig_path=f'ana/results_report/images/el_waveform/{png}'


#fig_evo[0].savefig(fig_path,dpi=600,format='svg')




In [ ]:
if phy_sig=='rsp_phase':
    png1='Aware - unaware inh'
    png2='Aware - unaware exh'
    color_1='tab:green'
    color_2='tab:pink'
elif phy_sig=='cardiac_phase':
    png1='Aware - unaware sys'
    png2='Aware - unaware dia'
    color_1='tab:orange'
    color_2='blue'



ts=tTest_ana([cond_1,cond_2],crop_value=crop_value,FDR=True,plot_times='peaks',averages=0.02,p_val=0.05,picks=picked,png=png1,color=color_1,effect_size='whole',axes=ax4)
ts=tTest_ana([cond_3,cond_4],crop_value=crop_value,FDR=True,plot_times='peaks',averages=0.02,p_val=0.05,picks=picked,png=png2,color=color_2,effect_size='whole',axes=ax4)

#plt.tight_layout()
#ts=tTest_ana([cond5,cond6],crop_value=data_shape,FDR=False,plot_times='peaks',averages=0.02,p_val=0.01,picks=picks_VAN,png=png3,plot_sig_lines=False)
#ts=tTest_ana([cond7,cond8],crop_value=data_shape,FDR=False,plot_times='peaks',averages=0.02,p_val=0.01,picks=picks_VAN,png=png4)
#plt.tight_layout()


png=f'{rsp_cond_lab}CACU_{phy_sig}_{region_ax1}_{region_ax2}.svg'
fig_path=f'ana/results_report/images/el_waveform/{png}'

fig_fin.savefig(fig_path,dpi=1200,format='svg')



In [ ]:
fig_fin.show()